In [1]:
import gym
import numpy as np
import matplotlib.pyplot as plt

env = gym.make('Acrobot-v1')
state = env.reset()

# set parameters
iterations = 2
samples = 25
elite = 7
iterations_cum_reward = []
# initialise mean and standard deviation
mu = np.random.uniform(size=state.shape)
sigma = np.random.uniform(low=0.001, size=state.shape)

def sample_gaussian(mu, sigma, n, m):
    weights = np.random.normal(loc=mu, scale=sigma, size=(n, m))
    return weights


def evaluate_binary_policy(env, theta, render=False):
    max_steps = 2000
    cum_reward = 0
    state = env.reset()
    for _ in range(max_steps):
        action = int(np.dot(theta, state) > 0)
        state, reward, terminal, info = env.step(action)
        cum_reward += reward
        if render:
            env.render()
        if terminal:
            break
    return cum_reward

C:\Users\marka\anaconda3\lib\site-packages\gym\core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
C:\Users\marka\anaconda3\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [2]:


def learn_ce():
    for _ in range(iterations):
        # Obtain n samples from current sampling distribution
            # Evaluate objective function at sampled points
            # Sort X by objective function values and take n elite samples
            # Update the parameters of sampling distribution
        iterations_cum_reward.append(cum_reward.mean())
    env.close()



In [3]:


env = gym.make('Acrobot-v1')
state = env.reset()
learn_ce()
plt.scatter(range(iterations), iterations_cum_reward)
plt.show()



NameError: name 'cum_reward' is not defined

In [5]:
import numpy as np
import gym
from gym.spaces import Discrete, Box
from gym import wrappers
#===================================
#Polices
#==================================

class DeterministicDiscreteActionLinearPolicy(object):
    def __init__(self, theta, ob_space, ac_space):
        """
        dim_ob: dimension of observations
        n_actions: number of actions
        theta: flat vector of parameters
        """
        dim_ob = ob_space.shape[0]
        n_actions = ac_space.n
        assert len(theta) == (dim_ob + 1) * n_actions
        self.W = theta[0: dim_ob * n_actions].reshape(dim_ob, n_actions)
        self.b = theta[dim_ob * n_actions : None].reshape(1,n_actions)

    def act(self, ob):
        """
        """
        y = ob.dot(self.W) + self.b
        a = y.argmax()
        return a

class DeterministicContinuousActionLinerPolicy(object):
    def __init__(self, theta, ob_space, ac_space):
        """
        dim_ob: dimension of observations
        dim_ac: dimension of action vector
        theta: flat vector of parameters
        """
        self.ac_space = ac_space
        dim_ob = ob_space.shape[0]
        dim_ac = ac_space.shape[0]
        assert len(theta) == (dim_ob + 1) * dim_ac
        self.W = theta[0 : dim_ob * dim_ac].reshape(dim_ob, dim_ac)
        self.b = theta[dim_ob * dim_ac : None]

    def act(self, ob):
        a = np.clip(ob.dot(self.W) + self.b, self.ac_space.low, self.ac_space.high)
        return a

def do_episode(policy, env, num_steps, render=False):
    total_rew = 0
    ob = env.reset()
    for t in range(num_steps):
        a = policy.act(ob)
        (ob, reward, done, _info) = env.step(a)
        total_rew += reward
        if render and t%3==0: env.render()
        if done: break
    return total_rew

env = None
def noisy_evaluation(theta):
    policy = make_policy(theta)
    rew = do_episode(policy, env, num_steps)
    return rew

def make_policy(theta):
    if isinstance(env.action_space, Discrete):
        return DeterministicDiscreteActionLinearPolicy(theta, env.observation_space, env.action_space)
    elif isinstance(env.action_space, Box):
        return DeterministicContimuousActionLinearPolicy(theta,env.observation_space, env.action_space)
    else:
        raise NotImplementedError

#task settings:
env = gym.make('Acrobot-v1')#change as needed

#env = wrappers.Monitor(env, "./tmp/CartPole-v0-experiment-1")
env = wrappers.Monitor(env, "./tmp/Acrobot-v1-experiment-6")
num_steps = 500#maximum length of episode
#alg settings:
n_iter = 100# number of iterations of CEM
batch_size = 25# number of samples per batch
elite_frac = 0.2# fraction of samples used as elite set

if isinstance(env.action_space, Discrete):
    dim_theta = (env.observation_space.shape[0] + 1) * env.action_space.n
elif isinstance(env.action_space, Box):
    dim_theta = (env.observation_space.shape[0] + 1) * env.action_space.shape[0]
else:
    raise NotImplementedError

#Initialize mean and standard deviation
theta_mean = np.zeros(dim_theta)
theta_std = np.ones(dim_theta)

#Now, for the algorithm
for iteration in xrange(n_iter):
    #Sample parameter vectors
    thetas = np.random.normal(theta_mean, theta_std, (batch_size,dim_theta))
    rewards = [noisy_evaluation(theta) for theta in thetas]
    #get elite parameters
    n_elite = int(batch_size * elite_frac)
    elite_inds = np.argsort(rewards)[batch_size - n_elite:batch_size]
    elite_thetas = [thetas[i] for i in elite_inds]
    #Update theta_mean, theta_std
    theta_mean = np.mean(elite_thetas,axis=0)
    theta_std = np.std(elite_thetas,axis=0)
    print ("iteration %i. mean f: %8.3g. max f: %8.3g" % (iteration,np.mean(rewards),np.max(rewards)))
    do_episode(make_policy(theta_mean), env, num_steps, render=True)

AttributeError: module 'gym.wrappers' has no attribute 'Monitor'